#### Using info from yuji96/reproduce-essay.ipynb

In [1]:
import textwrap
import re
import pandas as pd

In [2]:
df = pd.read_csv("../data/train_logs.csv")

In [3]:
def print_essay(s, width=100):
    print(
        "\n".join(
            textwrap.wrap(
                s, width=width, break_long_words=False, replace_whitespace=False
            )
        )
    )

In [4]:
def extract_essay(df):
    essay=''
    pos=0

    print(df)

    for _, row in df.iterrows():
        match row["activity"]:
            case "Nonproduction":
                pass
            case "Input" | "Paste":
                # qqqq|q - ZZZ -> qqqqZZZ|q
                essay = essay[:pos] + row["text_change"] + essay[pos:]
            case "Remove/Cut":
                # qqqqZZZ|q - ZZZ -> qqqq|q
                essay = essay[: pos - len(row["text_change"])] + essay[pos:]
            case "Replace":
                # qqZZZ|qqq -> ZZZ-YYY -> qqYYY|qqq
                # The replacement position seems to always be just before the cursor.
                before, after = row["text_change"].split(" => ")
                essay = essay[: pos - len(before)] + after + essay[pos:]
            case _:
                # Move
                if row["activity"].startswith("Move"):
                    # qq|ZZZ|q -> qq|AAA|q -> q|ZZZ|q|AAA|q -> q|ZZZ|qq
                    a, b, c, d = map(
                        int,
                        re.match(
                            r"Move From \[(\d+), (\d+)\] To \[(\d+), (\d+)\]",
                            row["activity"],
                        ).groups(),
                    )
                    moving = essay[a:b]
                    essay = essay[:a] + "A" * (b - a) + essay[b:]
                    essay = essay[:c] + moving + essay[c:]
                    essay = essay.replace("A", "")
                # Error
                else:
                    raise

    return essay


SyntaxError: invalid syntax (1527211236.py, line 8)

In [ ]:
test = df.groupby('id').apply(extract_essay)

MemoryError: 